In [2]:
import pandas as pd
import numpy as np

### Code to combine two sentiment fragment csvs

In [110]:
# df_most = pd.read_csv('../team_data/all_tweets_sentiment.csv', engine='python')
# df_last = pd.read_csv('../team_data/all_tweets_sentiment_2.csv', engine='python')
# df_all = pd.concat([df_most, df_last], ignore_index=True)
# df_all = df_all.drop(['index'], axis=1).set_index(['week','team'])
# df_all

In [3]:
df_all = pd.read_csv('../team_data/all_sentiment_combined.csv')
df_all = df_all.set_index(['week','team'])
df_all


timestamp  \
week team                                 
12   49ers    2022-11-27 20:24:43+00:00   
     49ers    2022-11-27 20:24:42+00:00   
     49ers    2022-11-27 20:24:03+00:00   
     49ers    2022-11-27 20:23:47+00:00   
     49ers    2022-11-27 20:23:38+00:00   
...                                 ...   
19   vikings  2023-01-11 20:33:58+00:00   
     vikings  2023-01-11 20:32:47+00:00   
     vikings  2023-01-11 20:32:00+00:00   
     vikings  2023-01-11 20:31:00+00:00   
     vikings  2023-01-11 20:30:00+00:00   

                                                     tweet_text  \
week team                                                         
12   49ers        1hr til kick off. Let’s Go Niners!! #fttb ❤️💛   
     49ers    Big test today for Jimmy Garoppolo and the #49...   
     49ers    #49ers have held three straight opponents scor...   
     49ers                @49erHodge Hell yea GO NINERS 🤘 #FTTB   
     49ers    #49ers are seeking their first 4-game regular-...   
...                                                         ...   
19   vikings  @heykayadams @UpAndAdamsShow @kgxix @Young_Sla...   
     vikings  Here’s the first #Giants injury report for the...   
     vikings  22 Things the 2022 Regular Season Revealed Abo...   
     vikings  Cook on The Running Game Heading Into Sunday: ...   
     vikings  What would be better than a #Vikings win this ...   

                           userid     label     score  
week team                                              
12   49ers    1404574279811878913  positive  0.983667  
     49ers               38617573   neutral  0.865918  
     49ers               38617573   neutral  0.501523  
     49ers              601772815  positive  0.939245  
     49ers               38617573   neutral  0.511218  
...                           ...       ...       ...  
19   vikings  1422946127750606853   neutral  0.587164  
     vikings           1561574120   neutral  0.929358  
     vikings           3035196396   neutral  0.879593  
     vikings           3035196396   neutral  0.539383  
     vikings             27704087  positive  0.961381  

[645042 rows x 5 columns]

### Check for NaN values

In [4]:
count_nan = df_all['tweet_text'].isnull()
nan_index = np.where(count_nan)[0].tolist()
df_all.iloc[nan_index]

,,timestamp,tweet_text,userid,label,score
week,team,,,,,


### get tuples of week team combinations

In [5]:
# wks = df_all.index.unique(level=0)
# tms = df_all.index.unique(level=1)
wks_teams = df_all.index.unique()
wks_teams[12]

(12, 'eagles')

In [6]:

def get_df_wk_team(df, wk, team):
    return df.xs((wk,team), level=[0,1])


def get_sentiment_counts(df):
    counts = df['label'].value_counts(sort=False)
    counts_dict = counts_to_dict(counts)
    return counts_dict

def counts_to_dict(counts):
    # print(counts)
    count_dict = {
        'neutral': counts['neutral'],
        'positive': counts['positive'],
        'negative': counts['negative']
    }
    return count_dict

def get_game_time(df):
    df = df.sort_values(['timestamp'], ascending=False)
    return df.iloc[0,0]

def get_24hr_df(df):
    df1 = df.copy()
    df1['timestamp'] = pd.to_datetime(df['timestamp'], format = '%Y-%m-%d %H:%M:%S.%f')
    game_time = get_game_time(df1)
    # return game_time
    game_time_24 = game_time - pd.to_timedelta('24:00:00')
    return df1.loc[(df1['timestamp'] >= game_time_24) & (df1['timestamp'] <= game_time)]


df_bears = get_df_wk_team(df_all, 12, 'bears')
df_bears
get_24hr_df(df_bears)

timestamp  \
week team                              
12   bears 2022-11-27 16:58:58+00:00   
     bears 2022-11-27 16:58:31+00:00   
     bears 2022-11-27 16:57:45+00:00   
     bears 2022-11-27 16:55:18+00:00   
     bears 2022-11-27 16:54:32+00:00   
...                              ...   
     bears 2022-11-26 17:04:46+00:00   
     bears 2022-11-26 17:04:10+00:00   
     bears 2022-11-26 17:02:05+00:00   
     bears 2022-11-26 16:59:41+00:00   
     bears 2022-11-26 16:59:41+00:00   

                                                   tweet_text  \
week team                                                       
12   bears  For the rest of the season, lol. #DaBears #BEA...   
     bears  🚨BREAKING NEWS: Justin Fields is officially ou...   
     bears  #Jets RB James Robinson is a healthy inactive ...   
     bears  AdamRonis: Justin Fields is inactive for the #...   
     bears  Now- my subscription is 7,77 cause I like it💦 ...   
...                                                       ...   
     bears  Yeah, Duh! Water is wet too! —-&gt; Robert Sal...   
     bears  Which game Ryan Poles at to start the day…. #D...   
     bears  Week 12 Fantasy Busts: Justin Fields, Cam Aker...   
     bears  ICYMI: Vegas odds for #Bears-Jets https://t.co...   
     bears  ICYMI: Vegas odds for #Bears-Jets https://t.co...   

                         userid     label     score  
week team                                            
12   bears   718550336575815680   neutral  0.606939  
     bears  1100098670324838403   neutral  0.496972  
     bears  1395552062650437635   neutral  0.887163  
     bears             11221412   neutral  0.829782  
     bears  1239072082077667328  positive  0.848296  
...                         ...       ...       ...  
     bears           2826875847   neutral  0.560948  
     bears             21418293   neutral  0.884996  
     bears           3013919629  negative  0.530575  
     bears  1283164327181197314   neutral  0.954360  
     bears  1042787770081714177   neutral  0.954360  

[839 rows x 5 columns]

In [7]:
def get_sentiment_counts_team_wk(df):
    sentiment_stats = []
    for data in wks_teams:
        week = data[0]
        team = data[1]

        team_df = get_df_wk_team(df, week, team)
        team_24_df = get_24hr_df(team_df)
        sentiment_counts_96 = get_sentiment_counts(team_df)
        sentiment_counts_24 = get_sentiment_counts(team_24_df)

        sentiment_stats.append({
            'team': data[1],
            'week': data[0],
            'positive96': sentiment_counts_96['positive'],
            'positive24': sentiment_counts_24['positive'],
            'neutral96': sentiment_counts_96['neutral'],
            'neutral24': sentiment_counts_24['neutral'],
            'negative96': sentiment_counts_96['negative'],
            'negative24': sentiment_counts_24['negative']
        })
    return sentiment_stats

all_sentiment_counts = get_sentiment_counts_team_wk(df_all)

In [8]:
all_sent_df = pd.DataFrame.from_dict(all_sentiment_counts).set_index(['week','team'])
# all_sent_df['sum'] = all_sent_df[list(all_sent_df.columns)].sum(axis=1)
all_sent_df

positive96  positive24  neutral96  neutral24  negative96  \
week team                                                                 
12   49ers           1165         449       1368        445         216   
     bears            563         204       1531        548         251   
     bengals          885         340       1098        422          96   
     broncos          671         197        804        241         134   
     browns           878         356       1315        527         423   
...                   ...         ...        ...        ...         ...   
19   giants          3100        1504       2097        683         403   
     jaguars         2370        1238       2429       1102         372   
     ravens           283          86        793        227         194   
     seahawks        1372         820       1596        741         253   
     vikings         4033        1903       2951       1173         713   

               negative24  
week team                  
12   49ers             90  
     bears             87  
     bengals           43  
     broncos           42  
     browns           195  
...                   ...  
19   giants           144  
     jaguars          140  
     ravens            47  
     seahawks         141  
     vikings          354  

[215 rows x 6 columns]

In [9]:
all_sent_df['sentiment_96h'] = (
    all_sent_df['positive96'] - all_sent_df['negative96']
    ) / (
        all_sent_df['positive96'] + all_sent_df['negative96'] + all_sent_df['neutral96']
    )
all_sent_df['sentiment_24h'] = (
    all_sent_df['positive24'] - all_sent_df['negative24']
    ) / (
        all_sent_df['positive24'] + all_sent_df['negative24'] + all_sent_df['neutral24']
    )
all_sent_df['swing'] = (all_sent_df['sentiment_24h'] - all_sent_df['sentiment_96h'])
# all_sent_df['swing'] = round(all_sent_df['swing']*100,2)
all_sent_df

positive96  positive24  neutral96  neutral24  negative96  \
week team                                                                 
12   49ers           1165         449       1368        445         216   
     bears            563         204       1531        548         251   
     bengals          885         340       1098        422          96   
     broncos          671         197        804        241         134   
     browns           878         356       1315        527         423   
...                   ...         ...        ...        ...         ...   
19   giants          3100        1504       2097        683         403   
     jaguars         2370        1238       2429       1102         372   
     ravens           283          86        793        227         194   
     seahawks        1372         820       1596        741         253   
     vikings         4033        1903       2951       1173         713   

               negative24  sentiment_96h  sentiment_24h     swing  
week team                                                          
12   49ers             90       0.345216       0.364837  0.019621  
     bears             87       0.133049       0.139452  0.006403  
     bengals           43       0.379509       0.368944 -0.010565  
     broncos           42       0.333748       0.322917 -0.010831  
     browns           195       0.173930       0.149351 -0.024579  
...                   ...            ...            ...       ...  
19   giants           144       0.481607       0.583441  0.101833  
     jaguars          140       0.386386       0.442742  0.056356  
     ravens            47       0.070079       0.108333  0.038255  
     seahawks         141       0.347408       0.398942  0.051535  
     vikings          354       0.431337       0.451603  0.020267  

[215 rows x 9 columns]

In [10]:
all_sent_df.xs((12,'49ers'), level=[0,1]).iloc[:, -3 :].values[0]

array([0.34521644, 0.3648374 , 0.01962096])

In [212]:
# all_sent_df.to_csv('../team_data/wk_tm_swing_sentiment.csv')

### Add Sentiment Values to Team Game Stats

In [12]:
wks_teams[:3]

MultiIndex([(12,   '49ers'),
            (12,   'bears'),
            (12, 'bengals')],
           names=['week', 'team'])

In [13]:
df_12 = pd.read_csv('../game_stats/wk_12_game_stats_clean.csv')

In [14]:
df_12.head()

,Week,Day,Date,home,away,home_score,away_score,away_spread,away_rush_att,away_rush_yds,...,home_pass_tds,home_pass_ints,away_sacks,away_sack_yds,home_sacks,home_sack_yds,home_penalty,home_penalty_yds,away_penalty,away_penalty_yds
0,12,Thu,2022-11-24,lions,bills,25,28,-9.5,29,164,...,2,0,3,16,2,10,3,32,5,36
1,12,Thu,2022-11-24,cowboys,giants,28,20,-10.0,21,90,...,2,2,3,18,0,0,13,86,7,66
2,12,Thu,2022-11-24,vikings,patriots,33,26,-2.5,13,45,...,3,1,3,18,1,9,3,20,6,55
3,12,Sun,2022-11-27,titans,bengals,16,20,-1.5,28,108,...,0,0,1,4,1,10,8,57,9,80
4,12,Sun,2022-11-27,browns,buccaneers,23,17,-3.0,20,96,...,1,1,3,17,4,32,5,42,9,70


In [49]:
# # for each row in df_wk

# # home_team = df['home']
# # away_team = df['away']
# # 
# # sentiment_96h	sentiment_24h	swing
# # home_values = all_sent_df.x s((wk,home_team), level=[0,1]).iloc[:, -3 :].values[0]
# # df['home_sent_96', 'home_sent_24', 'home_swing'] = home_values
# # away_values = all_sent_df.x s((wk,away_team), level=[0,1]).iloc[:, -3 :].values[0]
# # df['away_sent_96', 'away_sent_24', 'away_swing'] = away_values

# columns_away = {
#     'positive96': 'away_positive96',
#     'positive24': 'away_positive24',
#     'neutral96': 'away_neutral96',
#     'neutral24': 'away_neutral24',
#     'negative96': 'away_negative96',
#     'negative24': 'away_negative24',
#     'sentiment_96h': 'away_sentiment_96h',
#     'sentiment_24h': 'away_sentiment_24h',
#     'swing': 'away_swing',
# }


# columns_home = {
#     'positive96': 'home_positive96',
#     'positive24': 'home_positive24',
#     'neutral96': 'home_neutral96',
#     'neutral24': 'home_neutral24',
#     'negative96': 'home_negative96',
#     'negative24': 'home_negative24',
#     'sentiment_96h': 'home_sentiment_96h',
#     'sentiment_24h': 'home_sentiment_24h',
#     'swing': 'home_swing',
# }

# def change_keys(dictionary, home_away):
#     for key in dictionary.keys():
#         print(key)


# def do_something(wk, sent_df):
#     df_wk = pd.read_csv(f'../game_stats/wk_{wk}_game_stats_clean.csv')
#     home_values = []
#     away_values = []
#     for row in df_wk.iloc[3:4].itertuples():
#         home_team = row.home
#         away_team = row.away
#         print('Home', home_team, '\t Away: ', away_team)
#         home_df = all_sent_df.xs((wk,home_team), level=[0,1])
#         home_df = home_df.rename(columns=columns_home)
#         home_records = home_df.to_dict('records')
#         away_df = all_sent_df.xs((wk,away_team), level=[0,1])
#         away_df = home_df.rename(columns=columns_away)
#         away_records = away_df.to_dict('records')
#         # if home_values == [] or away_values == []:
#         #     home_values = [[0], [0], [0]]
#         #     away_values = [[0], [0], [0]]
#         print(home_records)
#         print(away_records)
#         records = home_records[0].update(away_records[0])
#         print(records)
#         # row['home_sent_96', 'home_sent_24', 'home_swing'] = home_values
#         # row['away_sent_96', 'away_sent_24', 'away_swing'] = away_values
        
        
# test_dict = {
#     'positive96': 472,
#     'positive24': 187,
#     'neutral96': 865,
#     'neutral24': 286,
#     'negative96': 147,
#     'negative24': 56,
#     'sentiment_96h': 0.21900269541778974,
#     'sentiment_24h': 0.24763705103969755,
#     'swing': 0.028634355621907803
#     }
# # for each row in df_wk

# # home_team = df['home']
# # away_team = df['away']
# # 
# # sentiment_96h	sentiment_24h	swing
# # home_values = all_sent_df.x s((wk,home_team), level=[0,1]).iloc[:, -3 :].values[0]
# # df['home_sent_96', 'home_sent_24', 'home_swing'] = home_values
# # away_values = all_sent_df.x s((wk,away_team), level=[0,1]).iloc[:, -3 :].values[0]
# # df['away_sent_96', 'away_sent_24', 'away_swing'] = away_values
empty_record = {
    'away_positive96': np.nan,
    'away_positive24': np.nan,
    'away_neutral96': np.nan,
    'away_neutral24': np.nan,
    'away_negative96': np.nan,
    'away_negative24': np.nan,
    'away_sentiment_96h': np.nan,
    'away_sentiment_24h': np.nan,
    'away_swing': np.nan,
    'home_positive96': np.nan,
    'home_positive24': np.nan,
    'home_neutral96': np.nan,
    'home_neutral24': np.nan,
    'home_negative96': np.nan,
    'home_negative24': np.nan,
    'home_sentiment_96h': np.nan,
    'home_sentiment_24h': np.nan,
    'home_swing': np.nan,
}


columns_away = {
    'positive96': 'away_positive96',
    'positive24': 'away_positive24',
    'neutral96': 'away_neutral96',
    'neutral24': 'away_neutral24',
    'negative96': 'away_negative96',
    'negative24': 'away_negative24',
    'sentiment_96h': 'away_sentiment_96h',
    'sentiment_24h': 'away_sentiment_24h',
    'swing': 'away_swing',
}


columns_home = {
    'positive96': 'home_positive96',
    'positive24': 'home_positive24',
    'neutral96': 'home_neutral96',
    'neutral24': 'home_neutral24',
    'negative96': 'home_negative96',
    'negative24': 'home_negative24',
    'sentiment_96h': 'home_sentiment_96h',
    'sentiment_24h': 'home_sentiment_24h',
    'swing': 'home_swing',
}

def change_keys(dictionary, home_away):
    for key in dictionary.keys():
        print(key)


def get_week_sentiment(wk, sent_df):
    df_wk = pd.read_csv(f'../game_stats/wk_{wk}_game_stats_clean.csv')
    all_records = []
    for row in df_wk.itertuples():
        home_team = row.home
        away_team = row.away
        
        print('Home', home_team, '\t Away: ', away_team)

        home_df = all_sent_df.xs((wk,home_team), level=[0,1])
        home_df = home_df.rename(columns=columns_home)
        home_records = home_df.to_dict('records')

        away_df = all_sent_df.xs((wk,away_team), level=[0,1])
        away_df = away_df.rename(columns=columns_away)
        away_records = away_df.to_dict('records')

        if home_records == [] or away_records == []:
            all_records.append(empty_record)
        else: 
            records = {**home_records[0], **away_records[0]}
            all_records.append(records)
    records_df = pd.DataFrame.from_dict(all_records)
    new_wk = pd.concat([df_wk, records_df], axis = 1)   
    new_wk.to_csv(f'../game_stats/wk_{wk}_game_stat_sent.csv')
    return new_wk
        
        

# get_week_sentiment(12, all_sent_df)


In [52]:
for i in range(13,20,1):
    # print(i)
    get_week_sentiment(i, all_sent_df)

Home patriots 	 Away:  bills
Home texans 	 Away:  browns
Home ravens 	 Away:  broncos
Home lions 	 Away:  jaguars
Home vikings 	 Away:  jets
Home giants 	 Away:  commanders
Home eagles 	 Away:  titans
Home falcons 	 Away:  steelers
Home bears 	 Away:  packers
Home 49ers 	 Away:  dolphins
Home rams 	 Away:  seahawks
Home bengals 	 Away:  chiefs
Home raiders 	 Away:  chargers
Home cowboys 	 Away:  colts
Home buccaneers 	 Away:  saints
Home rams 	 Away:  raiders
Home bills 	 Away:  jets
Home bengals 	 Away:  browns
Home cowboys 	 Away:  texans
Home lions 	 Away:  vikings
Home titans 	 Away:  jaguars
Home giants 	 Away:  eagles
Home steelers 	 Away:  ravens
Home broncos 	 Away:  chiefs
Home seahawks 	 Away:  panthers
Home 49ers 	 Away:  buccaneers
Home chargers 	 Away:  dolphins
Home cardinals 	 Away:  patriots
Home seahawks 	 Away:  49ers
Home vikings 	 Away:  colts
Home browns 	 Away:  ravens
Home bills 	 Away:  dolphins
Home saints 	 Away:  falcons
Home panthers 	 Away:  steelers
Home b